In [1]:
import pandas as pd
import numpy as np

from scipy.spatial import cKDTree

import os

import netCDF4 as nc
import xarray as xr

import re
from glob import glob

import time

In [2]:
file_path = '../01_data/01_biological_data'
file_name = 'metadata.tsv'

sat_data_path = '../01_data/00_satellite_data'

file = os.path.join(file_path, file_name)
md = pd.read_csv(file, sep='\t', index_col=0)

md_srf = md[md.Layer == 'SRF'].copy()

md_srf['Event.date.YYYYMM'] = md_srf['Event.date'].str[:7].str.replace('-', '')
md_srf['Event.date.YYYYMM01'] = md_srf['Event.date'].str[:7].str.replace('-', '')+'01'
#print(md_srf[['Event.date', 'Event.date.YYYYMM', 'Event.date.YYYYMM01']])

# Select nearest

In [71]:
sat_data_path = '../../01_data/00_satellite_data'
satellite_features = [
    'CHL.chlor_a', 'FLH.nflh', 'KD.Kd_490', 'PAR.par', 'PIC.pic', 'POC.poc',
    'RRS.Rrs_412', 'RRS.Rrs_443', 'RRS.Rrs_469', 'RRS.Rrs_488', 'RRS.Rrs_531',
    'RRS.Rrs_547', 'RRS.Rrs_555', 'RRS.Rrs_645', 'RRS.Rrs_667', 'RRS.Rrs_678', 'SST.sst'
]

satellite_data_terra = pd.DataFrame(index=md_srf.index, columns=satellite_features)
satellite_data_aqua = pd.DataFrame(index=md_srf.index, columns=satellite_features)

def find_satellite_file(directory, pattern):
    regex = re.compile(pattern)
    for file in os.listdir(directory):
        if regex.match(file):
            return os.path.join(directory, file)
    return None

start_time = time.time() # time start
for index, row in md_srf.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    date = row['Event.date.YYYYMM']
    
    # Read respective sat data
    for feature in satellite_features:
        resolution = '9km'
        
        pattern_terra = rf"TERRA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
        file_path_terra = find_satellite_file(sat_data_path, pattern_terra)
        pattern_aqua = rf"AQUA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
        file_path_aqua = find_satellite_file(sat_data_path, pattern_aqua)
        
        if file_path_terra and file_path_aqua:
            ds_terra = xr.open_dataset(file_path_terra)
            ds_aqua = xr.open_dataset(file_path_aqua)
            
            # select nearest for latitud & longitud
            try:
                data_point_terra = ds_terra[feature.split('.')[1]].sel(lat=latitude, lon=longitude, method='nearest')
                satellite_data_terra.at[index, feature] = data_point_terra.values
                data_point_aqua = ds_aqua[feature.split('.')[1]].sel(lat=latitude, lon=longitude, method='nearest')
                satellite_data_aqua.at[index, feature] = data_point_aqua.values

            except KeyError:
                print(f"Feature {feature} not found in dataset")
        else:
            if not file_path_terra:
                print(f"No TERRA file found for pattern: {pattern_terra}")
            if not file_path_aqua:
                print(f"No AQUA file found for pattern: {pattern_aqua}")

# avg between terra and aqua
satellite_data_avg = (satellite_data_terra.astype(float) + satellite_data_aqua.astype(float)) / 2

end_time = time.time() # end time
execution_time = end_time - start_time # total time


print("satellite nan values")
print(satellite_data_avg.isna().sum())
#print("TERRA Data:")
#print(satellite_data_terra)
#print("\nAQUA Data:")
#print(satellite_data_aqua)
#print("\nAverage Data:")
#print(satellite_data_avg)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")

satellite nan values
CHL.chlor_a    7
FLH.nflh       8
KD.Kd_490      7
PAR.par        2
PIC.pic        8
POC.poc        7
RRS.Rrs_412    7
RRS.Rrs_443    7
RRS.Rrs_469    7
RRS.Rrs_488    7
RRS.Rrs_531    7
RRS.Rrs_547    7
RRS.Rrs_555    7
RRS.Rrs_645    7
RRS.Rrs_667    7
RRS.Rrs_678    7
SST.sst        1
dtype: int64
Tiempo de ejecución: 19.33 segundos


# Select nearest non nan value

In [5]:
sat_data_path = '../01_data/00_satellite_data'
satellite_features = [
    'CHL.chlor_a', 'FLH.nflh', 'KD.Kd_490', 'PAR.par', 'PIC.pic', 'POC.poc',
    'RRS.Rrs_412', 'RRS.Rrs_443', 'RRS.Rrs_469', 'RRS.Rrs_488', 'RRS.Rrs_531',
    'RRS.Rrs_547', 'RRS.Rrs_555', 'RRS.Rrs_645', 'RRS.Rrs_667', 'RRS.Rrs_678', 'SST.sst'
]

satellite_data_terra = pd.DataFrame(index=md_srf.index, columns=satellite_features)
satellite_data_aqua = pd.DataFrame(index=md_srf.index, columns=satellite_features)

def find_satellite_file(directory, pattern):
    regex = re.compile(pattern)
    for file in os.listdir(directory):
        if regex.match(file):
            return os.path.join(directory, file)
    return None

def select_nearest_valid(ds, feature, latitude, longitude):
    data_point = ds[feature].sel(lat=latitude, lon=longitude, method='nearest')
    if not np.isnan(data_point.values):
        return data_point.values
    
    latitudes = ds['lat'].values
    longitudes = ds['lon'].values
    
    # Crear una cuadrícula de latitudes y longitudes
    lat_grid, lon_grid = np.meshgrid(latitudes, longitudes, indexing='ij')
    distances = np.sqrt((lat_grid - latitude)**2 + (lon_grid - longitude)**2)
    
    sorted_indices = np.unravel_index(np.argsort(distances, axis=None), distances.shape)

    for i in range(len(sorted_indices[0])):
        lat_idx = sorted_indices[0][i]
        lon_idx = sorted_indices[1][i]
        data_point = ds[feature].isel(lat=lat_idx, lon=lon_idx)
        if not np.isnan(data_point.values):
            return data_point.values
    
    return np.nan

start_time = time.time() # time start

for index, row in md_srf.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    date = row['Event.date.YYYYMM']
    
    # Read respective sat data
    for feature in satellite_features:
        resolution = '9km'
        
        pattern_terra = rf"TERRA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
        file_path_terra = find_satellite_file(sat_data_path, pattern_terra)
        pattern_aqua = rf"AQUA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
        file_path_aqua = find_satellite_file(sat_data_path, pattern_aqua)
        
        if file_path_terra and file_path_aqua:
            ds_terra = xr.open_dataset(file_path_terra)
            ds_aqua = xr.open_dataset(file_path_aqua)
            # select nearest non value for latitud and longitud
            try:
                variable_name = feature.split('.')[1]
                
                data_point_terra = select_nearest_valid(ds_terra, variable_name, latitude, longitude)
                satellite_data_terra.at[index, feature] = data_point_terra

                data_point_aqua = select_nearest_valid(ds_aqua, variable_name, latitude, longitude)
                satellite_data_aqua.at[index, feature] = data_point_aqua

            except KeyError:
                print(f"Feature {feature} not found in dataset")
        else:
            if not file_path_terra:
                print(f"No TERRA file found for pattern: {pattern_terra}")
            if not file_path_aqua:
                print(f"No AQUA file found for pattern: {pattern_aqua}")
                
# avg between terra and aqua
satellite_data_avg = (satellite_data_terra.astype(float) + satellite_data_aqua.astype(float)) / 2

end_time = time.time() # end time
execution_time = end_time - start_time # total time

print("satellite nan values")
print(satellite_data_avg.isna().sum())
#print("TERRA Data:")
#print(satellite_data_terra)
#print("\nAQUA Data:")
#print(satellite_data_aqua)
#print("\nAverage Data:")
#print(satellite_data_avg)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")

satellite nan values
CHL.chlor_a    0
FLH.nflh       0
KD.Kd_490      0
PAR.par        0
PIC.pic        0
POC.poc        0
RRS.Rrs_412    0
RRS.Rrs_443    0
RRS.Rrs_469    0
RRS.Rrs_488    0
RRS.Rrs_531    0
RRS.Rrs_547    0
RRS.Rrs_555    0
RRS.Rrs_645    0
RRS.Rrs_667    0
RRS.Rrs_678    0
SST.sst        0
dtype: int64
Tiempo de ejecución: 410.69 segundos


# Select $n$ nearest non values

In [3]:
sat_data_path = '../01_data/00_satellite_data'
satellite_features = [
    'CHL.chlor_a', 'FLH.nflh', 'KD.Kd_490', 'PAR.par', 'PIC.pic', 'POC.poc',
    'RRS.Rrs_412', 'RRS.Rrs_443', 'RRS.Rrs_469', 'RRS.Rrs_488', 'RRS.Rrs_531',
    'RRS.Rrs_547', 'RRS.Rrs_555', 'RRS.Rrs_645', 'RRS.Rrs_667', 'RRS.Rrs_678', 'SST.sst'
]

satellite_data_terra = pd.DataFrame(index=md_srf.index, columns=satellite_features)
satellite_data_aqua = pd.DataFrame(index=md_srf.index, columns=satellite_features)

def find_satellite_file(directory, pattern):
    regex = re.compile(pattern)
    for file in os.listdir(directory):
        if regex.match(file):
            return os.path.join(directory, file)
    return None

def select_n_nearest_valid(ds, feature, latitude, longitude, n=5):
    latitudes = ds['lat'].values
    longitudes = ds['lon'].values
    
    lat_grid, lon_grid = np.meshgrid(latitudes, longitudes, indexing='ij')
    distances = np.sqrt((lat_grid - latitude)**2 + (lon_grid - longitude)**2)
    
    sorted_indices = np.unravel_index(np.argsort(distances, axis=None), distances.shape)
    
    valid_points = []
    for i in range(len(sorted_indices[0])):
        lat_idx = sorted_indices[0][i]
        lon_idx = sorted_indices[1][i]
        data_point = ds[feature].isel(lat=lat_idx, lon=lon_idx)
        if not np.isnan(data_point.values):
            valid_points.append(data_point.values)
        if len(valid_points) >= n:
            break
    
    if len(valid_points) > 0:
        return np.mean(valid_points)
    else:
        return np.nan

start_time = time.time()

for index, row in md_srf.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    date = row['Event.date.YYYYMM']
    
    for feature in satellite_features:
        resolution = '9km'
        
        pattern_terra = rf"TERRA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
        file_path_terra = find_satellite_file(sat_data_path, pattern_terra)
        pattern_aqua = rf"AQUA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
        file_path_aqua = find_satellite_file(sat_data_path, pattern_aqua)
        
        if file_path_terra and file_path_aqua:
            ds_terra = xr.open_dataset(file_path_terra)
            ds_aqua = xr.open_dataset(file_path_aqua)
            try:
                variable_name = feature.split('.')[1]
                
                data_point_terra = select_n_nearest_valid(ds_terra, variable_name, latitude, longitude, n=9)
                satellite_data_terra.at[index, feature] = data_point_terra

                data_point_aqua = select_n_nearest_valid(ds_aqua, variable_name, latitude, longitude, n=9)
                satellite_data_aqua.at[index, feature] = data_point_aqua

            except KeyError:
                print(f"Feature {feature} not found in dataset")
        else:
            if not file_path_terra:
                print(f"No TERRA file found for pattern: {pattern_terra}")
            if not file_path_aqua:
                print(f"No AQUA file found for pattern: {pattern_aqua}")

satellite_data_avg = (satellite_data_terra.astype(float) + satellite_data_aqua.astype(float)) / 2

end_time = time.time()
execution_time = end_time - start_time

print("satellite nan values")
print(satellite_data_avg.isna().sum())

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")

satellite nan values
CHL.chlor_a    0
FLH.nflh       0
KD.Kd_490      0
PAR.par        0
PIC.pic        0
POC.poc        0
RRS.Rrs_412    0
RRS.Rrs_443    0
RRS.Rrs_469    0
RRS.Rrs_488    0
RRS.Rrs_531    0
RRS.Rrs_547    0
RRS.Rrs_555    0
RRS.Rrs_645    0
RRS.Rrs_667    0
RRS.Rrs_678    0
SST.sst        0
dtype: int64
Tiempo de ejecución: 2775.06 segundos


In [5]:
satellite_data_avg.to_csv('matrix_tara_old_adj_grids_09.tsv', sep='\t')

# Select nearest non nan value, deleting

In [12]:
sat_data_path = '../01_data/00_satellite_data'
satellite_features = [
    'CHL.chlor_a', 'FLH.nflh', 'KD.Kd_490', 'PAR.par', 'PIC.pic', 'POC.poc',
    'RRS.Rrs_412', 'RRS.Rrs_443', 'RRS.Rrs_469', 'RRS.Rrs_488', 'RRS.Rrs_531',
    'RRS.Rrs_547', 'RRS.Rrs_555', 'RRS.Rrs_645', 'RRS.Rrs_667', 'RRS.Rrs_678', 'SST.sst'
]

satellite_data_terra = pd.DataFrame(index=md_srf.index, columns=satellite_features)
satellite_data_aqua = pd.DataFrame(index=md_srf.index, columns=satellite_features)

def find_satellite_file(directory, pattern):
    regex = re.compile(pattern)
    for file in os.listdir(directory):
        if regex.match(file):
            return os.path.join(directory, file)
    return None

def select_nearest_valid(ds, feature, latitude, longitude, search_radius=0.5):
    latitudes = ds['lat'].values
    longitudes = ds['lon'].values
    
    lat_min = latitude - search_radius
    lat_max = latitude + search_radius
    lon_min = longitude - search_radius
    lon_max = longitude + search_radius
    
    ds_subset = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
    
    lat_vals = ds_subset['lat'].values
    lon_vals = ds_subset['lon'].values
    
    # Create a grid of distances
    distances = np.sqrt((lat_vals[:, np.newaxis] - latitude)**2 + (lon_vals[np.newaxis, :] - longitude)**2)
    sorted_distances = np.argsort(distances, axis=None)
    
    for idx in sorted_distances:
        lat_idx, lon_idx = np.unravel_index(idx, distances.shape)
        data_point = ds_subset[feature].isel(lat=lat_idx, lon=lon_idx)
        if not np.isnan(data_point.values):
            return data_point.values
    
    return np.nan

start_time = time.time() # time start

for index, row in md_srf.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    date = row['Event.date.YYYYMM']
    
    # Read respective sat data
    for feature in satellite_features:
        resolution = '9km'
        
        pattern_terra = rf"TERRA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
        file_path_terra = find_satellite_file(sat_data_path, pattern_terra)
        pattern_aqua = rf"AQUA_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
        file_path_aqua = find_satellite_file(sat_data_path, pattern_aqua)
        
        if file_path_terra and file_path_aqua:
            ds_terra = xr.open_dataset(file_path_terra)
            ds_aqua = xr.open_dataset(file_path_aqua)
            # select nearest non value for latitud and longitud
            try:
                variable_name = feature.split('.')[1]
                
                data_point_terra = select_nearest_valid(ds_terra, variable_name, latitude, longitude)
                satellite_data_terra.at[index, feature] = data_point_terra

                data_point_aqua = select_nearest_valid(ds_aqua, variable_name, latitude, longitude)
                satellite_data_aqua.at[index, feature] = data_point_aqua

            except KeyError:
                print(f"Feature {feature} not found in dataset")
        else:
            if not file_path_terra:
                print(f"No TERRA file found for pattern: {pattern_terra}")
            if not file_path_aqua:
                print(f"No AQUA file found for pattern: {pattern_aqua}")
                
# avg between terra and aqua
satellite_data_avg = (satellite_data_terra.astype(float) + satellite_data_aqua.astype(float)) / 2

end_time = time.time() # end time
execution_time = end_time - start_time # total time

print("satellite nan values")
print(satellite_data_avg.isna().sum())
#print("TERRA Data:")
#print(satellite_data_terra)
#print("\nAQUA Data:")
#print(satellite_data_aqua)
#print("\nAverage Data:")
#print(satellite_data_avg)

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")

satellite nan values
CHL.chlor_a    83
FLH.nflh       83
KD.Kd_490      83
PAR.par        83
PIC.pic        83
POC.poc        83
RRS.Rrs_412    83
RRS.Rrs_443    83
RRS.Rrs_469    83
RRS.Rrs_488    83
RRS.Rrs_531    83
RRS.Rrs_547    83
RRS.Rrs_555    83
RRS.Rrs_645    83
RRS.Rrs_667    83
RRS.Rrs_678    83
SST.sst        83
dtype: int64
Tiempo de ejecución: 14.25 segundos


# Test

In [39]:
def find_satellite_file(directory, pattern):
    regex = re.compile(pattern)
    for file in os.listdir(directory):
        if regex.match(file):
            return os.path.join(directory, file)
    return None

date='200909'
feature = 'FLH.nflh' 
resolution = '9km'

pattern = rf"{satellite_name}_MODIS\.{date}01_{date}\d{{2}}\.L3m\.MO\.{feature}\.{resolution}\.nc"
file_path = find_satellite_file(sat_data_path, pattern)

ds = xr.open_dataset(file_path)

In [69]:
satellite_data_avg.isna().sum()
satellite_data_avg

,CHL.chlor_a,FLH.nflh,KD.Kd_490,PAR.par,PIC.pic,POC.poc,RRS.Rrs_412,RRS.Rrs_443,RRS.Rrs_469,RRS.Rrs_488,RRS.Rrs_531,RRS.Rrs_547,RRS.Rrs_555,RRS.Rrs_645,RRS.Rrs_667,RRS.Rrs_678,SST.sst
TSC_NAME,,,,,,,,,,,,,,,,,
TSC001,0.320456,0.072980,0.0514,43.424000,0.000041,83.399902,0.004447,0.004494,0.004600,0.004193,0.002313,0.001832,0.001606,0.000090,0.000107,0.000140,22.905000
TSC003,0.138729,0.050837,0.0330,39.139997,0.000011,43.199951,0.007104,0.006388,0.005885,0.004918,0.002085,0.001565,0.001351,0.000129,0.000153,0.000172,25.375000
TSC005,0.078143,0.047820,0.0279,40.142998,0.000017,34.299805,0.010119,0.008799,0.007673,0.006127,0.002323,0.001727,0.001472,0.000140,0.000173,0.000187,26.132499
TSC008,0.167823,0.056745,0.0301,24.141998,0.000011,42.699951,0.005551,0.005221,0.005035,0.004234,0.001719,0.001267,0.001112,0.000004,0.000059,0.000090,19.942499
TSC013,0.292031,0.074510,0.0489,16.330997,0.000010,78.599854,0.003375,0.003373,0.003564,0.003343,0.001850,0.001439,0.001303,0.000048,0.000154,0.000186,16.544999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSC276,0.500157,0.088323,0.0707,7.869997,0.000349,118.000000,0.005146,0.004813,0.004952,0.004923,0.003461,0.002879,0.002622,0.000250,0.000449,0.000459,0.280000
TSC280,0.167320,0.092020,0.0584,9.130997,0.000528,70.599854,0.011098,0.008550,0.007384,0.006434,0.004046,0.003200,0.002854,0.000332,0.000547,0.000549,-0.065000
TSC281,0.574612,0.082415,0.0771,12.114998,0.000211,140.899902,0.004376,0.003948,0.004175,0.004273,0.003180,0.002669,0.002387,0.000227,0.000376,0.000390,2.592500


In [62]:
for index, row in md_srf.iterrows():
    print(f"\nIndex: {index}")
    print(row)


Index: TSC001
PANGAEA sample id                                     TARA_Y200000002
Station.label                                                TARA_004
Layer                                                             SRF
Layer2                                                            EPI
polar                                                       non polar
lower.size.fraction                                              0.22
upper.size.fraction                                               1.6
Event.date                                           2009-09-15 11:30
Latitude                                                      36.5533
Longitude                                                     -6.5669
Depth.nominal                                                       9
Depth.Mixed.Layer                                                  12
Temperature                                                 20.238088
Oxygen                                                            NaN
Chlor